In [1]:
#Importing all necessary libraries / modules (in random order, but all of them are needed)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time, re
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

In [99]:
# Connect with the necessary Firestore and setup serive account for upload data into Google Cloud Database

from google.cloud import firestore

# Replace with the path to your service account JSON file
#service_account_json= "/Users/liangzhao/Desktop/Projects/my-project-hotel-database-9db570332d9e.json"
db = firestore.Client.from_service_account_json(service_account_json)


In [115]:
db.collection("Fall_Destination")

### 1. For Orlando ###
- use the selenimum to get the virtual chrome access 

In [26]:
## For Orlando##
#################
# Create a new instance of a browser
driver = webdriver.Chrome()

# Navigate to the website
url = "https://www.booking.com/"
driver.get(url)

# Define the aria-label attribute for the button
button_aria_label = "Dismiss sign-in info."

# Wait for the button in the pop-up to be clickable
wait = WebDriverWait(driver, 10)
popup_button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@aria-label="{button_aria_label}"]')))

# Click the button
popup_button.click()

# Wait for a moment after dismissing the popup
time.sleep(2)


# Modify the code to search for a location like New York
# Search for the first location: San Diego
# Locate the search input element by its name attribute
search_input = driver.find_element(By.NAME, 'ss')
# Clear any existing value and enter the search query
search_input.clear()
search_input.send_keys("Orlando")
# Wait for the search results to load (adjust the time if needed)
time.sleep(5)
#need to spend time to wait otherwise the webstie will automate eliminate your serach result for destination
# Simulate pressing the Enter key to submit the search query
search_input.send_keys(Keys.RETURN)


#select check-in date as September 1, 2023
driver.find_element(By.CSS_SELECTOR, "span[aria-label='1 September 2023']").click()

# select check-out date as September 5, 2023
driver.find_element(By.CSS_SELECTOR, "span[aria-label='5 September 2023']").click()

# click the search button
driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()



- B. Get the URL after search result - Orlando

In [ ]:
url = driver.current_url
type(url)
url


- C. Save each page html for reference (minimize the impact of website)

In [28]:
from bs4 import BeautifulSoup
import requests
import re
import random
from pymongo import MongoClient

# Add a user_agent header
user_agent = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36"}

save_directory = "/Users/liangzhao/Desktop/Projects/Booking_Hotel_Database/"

url = driver.current_url
# Get the current URL of the driver
url = driver.current_url

# Use a for loop to loop through pages 1 to 5 (25 results per page)
for num in range(0, 125, 25):
    URL = url + "&offset=" + str(num)  # Use the 'url' variable here
    source = requests.get(URL, headers=user_agent)
    soup = BeautifulSoup(source.content, "html.parser")
    
    # Save each page into a single file
    file_name = "OL_hotel_search_page_" + str(int((num + 25) / 25)) + ".html"
    file_path = save_directory + file_name
    
    with open(file_path, "w", encoding='utf-8') as file:
        file.write(str(soup))
    
    wait_time = random.randrange(10, 20)
    print("Done with page number " + str(int((num + 25) / 25)))
    
    # Wait a random time from 10 to 20 seconds before accessing the website again
    time.sleep(wait_time)

print("All done!")


Done with page number 1
Done with page number 2
Done with page number 3
Done with page number 4
Done with page number 5
All done!


- D. Get the Major Metrics: Price, Hotel Address, Hotel total review, Hotel distance, etc;

In [119]:
list1=[]
dat=0
for num in range(1, 6):
    file_path = "/Users/liangzhao/Desktop/projects/Booking_Hotel_Database/OL_hotel_search_page_"+str(num)+".html"
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()
        
    # Parse file as a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
    # find all the information for each hotel 
    
    #each property card will be changed due to the website 
        all_hotel_info = soup.find_all('div', {'class':'c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 da89aeb942'})

        for hotel_info in all_hotel_info:  
            #1. name for each hotel,notice brand new page will be change their class from website
            all_hotel_names = hotel_info.find_all('div', {'class': 'f6431b446c a23c043802'})
            for names in all_hotel_names:
                hotel_name = names.text
                #print(hotel_name)

            #2. links for each hotel
            all_hotel_links = hotel_info.find_all('a', {'class': 'e13098a59f'})             
            for links in all_hotel_links:
                hotel_link =  links.get('href')
                hotel_web = requests.get(hotel_link, headers = user_agent)
                #print(hotel_link)
                
                #2-1: hotel address
                hotel_soup = BeautifulSoup(hotel_web.content, 'html.parser')
                hotel_add = hotel_soup.find('span', {'class': 'hp_address_subtitle js-hp_address_subtitle jq_tooltip'})
                hotel_address = hotel_add.text.strip()
                    
            #3. price for 5 nights
            all_hotel_price = hotel_info.find_all('span', {'class':'f6431b446c fbd1d3018c e729ed5ab6'})
            for prices in all_hotel_price:
                sign, prices = prices.text.split("$", 1)
                five_night_prices = prices
                #print(five_night_prices)
                    
            #4. Review scores for each hotel
            all_hotel_scores = hotel_info.find_all('div', {'class': 'a3b8729ab1 d86cee9b25'})
            if all_hotel_scores:
                hotel_score = all_hotel_scores[0].text
            else:
                hotel_score = "No Score"
            #print(hotel_score)
            
            
            #5. number of reviews for each hotel
            all_hotel_reviews = hotel_info.find_all('div', {'class': 'abf093bdfe f45d8e4c32 d935416c47'})
            # Check if reviews element is found
            if all_hotel_reviews:
                # Extract the number of reviews from the text
                reviews_text = all_hotel_reviews[0].text
                total_reviews = reviews_text.split()[0]
            else:
                total_reviews = "No reviews"
            #print(total_reviews)
            
            #6. check if breakfast is included
            check_breakfast = hotel_info.find('span', {'class': 'e05969d63d'})
            try:
                for breakfast in check_breakfast:
                    hotel_breakfast = 1
            except TypeError:
                hotel_breakfast = 0
                
             #7. check if hotel allows free cancellation
            check_cancellation = hotel_info.find('div', {'class': 'd506630cf3'})
            try:
                for free_cancel in check_cancellation:
                    hotel_free_cancel = "Yes"
            except TypeError:
                hotel_free_cancel = "NO"
            #print("hotel_free_cancel:", hotel_free_cancel)
            
            #8. distance from center
            all_hotel_distance = hotel_info.find_all('span', {'class':'f419a93f12'})
            for distance in all_hotel_distance:
                if "from" in distance.text:
                    dis, from_center = distance.text.split(" ", 1)
                    dis_from_center = dis
                    #print(dis_from_center)
            dat+=1

                # store the info for hotels into a dictionary
                 # Create a dictionary for the current hotel
            hotel_dict = {
                "_id": dat,
                "Location": "Orlando",
                "hotel_name": hotel_name,
                "hotel_link": hotel_link,
                "hotel_address": hotel_address,
                "hotel_distance_from_center": dis_from_center,
                "five_nights_prices": five_night_prices,
                "hotel_score": hotel_score,
                "hotel_total_reviews": total_reviews,
                "free_breakfast_option": hotel_breakfast,
                "free_cancellation_option": hotel_free_cancel}
            list1.append(hotel_dict)
            
            db.collection("Fall_Destination").add(hotel_dict)



In [120]:
import pandas as pd
df=pd.DataFrame(list1)
df

,_id,Location,hotel_name,hotel_link,hotel_address,hotel_distance_from_center,five_nights_prices,hotel_score,hotel_total_reviews,free_breakfast_option,free_cancellation_option
0,1,Orlando,Grand Hotel Kissimmee at Celebration,https://www.booking.com/hotel/us/wingate-by-wy...,"3104 Parkway Blvd, Celebration, Orlando, 34747...",16.3,269,7.9,"4,987",0,NO
1,2,Orlando,SpringHill Suites by Marriott Orlando Lake Bue...,https://www.booking.com/hotel/us/spring-hill-m...,"8601 Vineland Avenue, Lake Buena Vista, Orland...",13.1,371,7.8,"3,501",1,NO
2,3,Orlando,Buena Vista Suites Orlando,https://www.booking.com/hotel/us/buena-vista-s...,"8203 World Center Drive, Lake Buena Vista, Orl...",14,725,8.8,"9,097",1,NO
3,4,Orlando,Renaissance Orlando at SeaWorld®,https://www.booking.com/hotel/us/renaissance-o...,"6677 Sea Harbor Drive, Orlando, FL 32821, Unit...",9.9,541,8.5,"1,307",0,NO
4,5,Orlando,Unforgettable Experience Luxury Vacation Home ...,https://www.booking.com/hotel/us/unforgettable...,"Celebration, Orlando, 34747, United States",21,942,No Score,No reviews,0,NO
...,...,...,...,...,...,...,...,...,...,...,...
121,122,Orlando,Fairfield Inn and Suites by Marriott Orlando N...,https://www.booking.com/hotel/us/fairfield-inn...,"5614 Vineland Road, Orlando, FL 32819, United ...",5.8,500,7.9,132,0,NO
122,123,Orlando,Beautiful Condo Near Disney World,https://www.booking.com/hotel/us/beautiful-con...,"7700 Westgate Boulevard, Celebration, Orlando,...",19.3,546,8.6,508,0,Yes
123,124,Orlando,Element Orlando International Drive,https://www.booking.com/hotel/us/element-orlan...,"5750 Central Florida Parkway, Orlando, 32821 F...",9.8,490,8.6,182,1,NO
124,125,Orlando,Econo Lodge International Drive,https://www.booking.com/hotel/us/international...,"7102 International Drive, Orlando, FL 32819, U...",7.6,302,6.2,"1,607",0,NO


### 2. For  Joshua Tree National Park ###

In [72]:
##############################
###Joshua Tree National Park##
##############################
# Create a new instance of a browser
driver = webdriver.Chrome()

# Navigate to the website
url = "https://www.booking.com/"
driver.get(url)

# Define the aria-label attribute for the button
button_aria_label = "Dismiss sign-in info."

# Wait for the button in the pop-up to be clickable
wait = WebDriverWait(driver, 10)
popup_button = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@aria-label="{button_aria_label}"]')))

# Click the button
popup_button.click()

# Wait for a moment after dismissing the popup
time.sleep(2)


# Modify the code to search for a location like New York
# Search for the first location: San Diego
# Locate the search input element by its name attribute
search_input = driver.find_element(By.NAME, 'ss')
# Clear any existing value and enter the search query
search_input.clear()
search_input.send_keys("Joshua Tree National Park")
# Wait for the search results to load (adjust the time if needed)
time.sleep(5)
#need to spend time to wait otherwise the webstie will automate eliminate your serach result for destination
# Simulate pressing the Enter key to submit the search query
search_input.send_keys(Keys.RETURN)


#select check-in date as September 1, 2023
driver.find_element(By.CSS_SELECTOR, "span[aria-label='1 September 2023']").click()

# select check-out date as September 5, 2023
driver.find_element(By.CSS_SELECTOR, "span[aria-label='5 September 2023']").click()

# click the search button
driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()



In [73]:
url = driver.current_url
type(url)
url


'https://www.booking.com/searchresults.html?ss=Joshua+Tree+National+Park&ssne=Joshua+Tree+National+Park&ssne_untouched=Joshua+Tree+National+Park&label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALL-s-mBsACAdICJDVkYjY2NzFjLTJhMzAtNDI2YS1hOWE0LWQ5OGQzYWI2MTY4OdgCBeACAQ&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=3102&dest_type=region&checkin=2023-09-01&checkout=2023-09-05&group_adults=2&no_rooms=1&group_children=0'

In [74]:
from bs4 import BeautifulSoup
import requests
import re
import random
from pymongo import MongoClient

# Add a user_agent header
user_agent = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36"}

save_directory = "/Users/liangzhao/Desktop/Projects/Booking_Hotel_Database/"

url = driver.current_url
# Get the current URL of the driver
url = driver.current_url

# Use a for loop to loop through pages 1 to 5 (25 results per page)
for num in range(0, 125, 25):
    URL = url + "&offset=" + str(num)  # Use the 'url' variable here
    source = requests.get(URL, headers=user_agent)
    soup = BeautifulSoup(source.content, "html.parser")
    
    # Save each page into a single file
    file_name = "Joshua_hotel_search_page_" + str(int((num + 25) / 25)) + ".html"
    file_path = save_directory + file_name
    
    with open(file_path, "w", encoding='utf-8') as file:
        file.write(str(soup))
    
    wait_time = random.randrange(10, 20)
    print("Done with page number " + str(int((num + 25) / 25)))
    
    # Wait a random time from 10 to 20 seconds before accessing the website again
    time.sleep(wait_time)

print("All done!")


Done with page number 1
Done with page number 2
Done with page number 3
Done with page number 4
Done with page number 5
All done!


In [125]:
list2=[]
dat=126
for num in range(1, 6):
    file_path = "/Users/liangzhao/Desktop/projects/Booking_Hotel_Database/Joshua_hotel_search_page_"+str(num)+".html"
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()
        
    # Parse file as a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
    # find all the information for each hotel 
    
    #each property card will be changed due to the website 
        all_hotel_info = soup.find_all('div', {'class':'c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 da89aeb942'})

        for hotel_info in all_hotel_info:  
            #1. name for each hotel,notice brand new page will be change their class from website
            all_hotel_names = hotel_info.find_all('div', {'class': 'f6431b446c a23c043802'})
            for names in all_hotel_names:
                hotel_name = names.text
                #print(hotel_name)

            #2. links for each hotel
            all_hotel_links = hotel_info.find_all('a', {'class': 'e13098a59f'})             
            for links in all_hotel_links:
                hotel_link =  links.get('href')
                hotel_web = requests.get(hotel_link, headers = user_agent)
                #print(hotel_link)
                
                #2-1: hotel address
                hotel_soup = BeautifulSoup(hotel_web.content, 'html.parser')
                hotel_add = hotel_soup.find('span', {'class': 'hp_address_subtitle js-hp_address_subtitle jq_tooltip'})
                hotel_address = hotel_add.text.strip()
                    
            #3. price for 5 nights
            all_hotel_price = hotel_info.find_all('span', {'class':'f6431b446c fbd1d3018c e729ed5ab6'})
            for prices in all_hotel_price:
                sign, prices = prices.text.split("$", 1)
                five_night_prices = prices
                #print(five_night_prices)
                    
            #4. Review scores for each hotel
            all_hotel_scores = hotel_info.find_all('div', {'class': 'a3b8729ab1 d86cee9b25'})
            if all_hotel_scores:
                hotel_score = all_hotel_scores[0].text
            else:
                hotel_score = "No Score"
            #print(hotel_score)
            
            
            #5. number of reviews for each hotel
            all_hotel_reviews = hotel_info.find_all('div', {'class': 'abf093bdfe f45d8e4c32 d935416c47'})
            # Check if reviews element is found
            if all_hotel_reviews:
                # Extract the number of reviews from the text
                reviews_text = all_hotel_reviews[0].text
                total_reviews = reviews_text.split()[0]
            else:
                total_reviews = "No reviews"
            #print(total_reviews)
            
            #6. check if breakfast is included
            check_breakfast = hotel_info.find('span', {'class': 'e05969d63d'})
            try:
                for breakfast in check_breakfast:
                    hotel_breakfast = 1
            except TypeError:
                hotel_breakfast = 0
                
             #7. check if hotel allows free cancellation
            check_cancellation = hotel_info.find('div', {'class': 'd506630cf3'})
            try:
                for free_cancel in check_cancellation:
                    hotel_free_cancel = "Yes"
            except TypeError:
                hotel_free_cancel = "NO"
            #print("hotel_free_cancel:", hotel_free_cancel)
            
            #8. distance from center
            all_hotel_distance = hotel_info.find_all('span', {'class':'f419a93f12'})
            for distance in all_hotel_distance:
                if "from" in distance.text:
                    dis, from_center = distance.text.split(" ", 1)
                    dis_from_center = dis
                    #print(dis_from_center)
            dat+=1

                # store the info for hotels into a dictionary
                 # Create a dictionary for the current hotel
            hotel_dict = {
                "_id": dat,
                "Location": "Joshua Tree National Park",
                "hotel_name": hotel_name,
                "hotel_link": hotel_link,
                "hotel_address": hotel_address,
                "hotel_distance_from_center": dis_from_center,
                "five_nights_prices": five_night_prices,
                "hotel_score": hotel_score,
                "hotel_total_reviews": total_reviews,
                "free_breakfast_option": hotel_breakfast,
                "free_cancellation_option": hotel_free_cancel}


            db.collection("Fall_Destination").add(hotel_dict)
            list2.append(hotel_dict)

            
            



In [127]:

df2=pd.DataFrame(list2)
df2

,_id,Location,hotel_name,hotel_link,hotel_address,hotel_distance_from_center,five_nights_prices,hotel_score,hotel_total_reviews,free_breakfast_option,free_cancellation_option
0,127,Joshua Tree National Park,Indian Wells Resort Hotel,https://www.booking.com/hotel/us/indian-wells-...,"76-661 Highway 111, Indian Wells, CA 92210, Un...",5.9,425,7.6,"2,196",0,NO
1,128,Joshua Tree National Park,Miracle Springs Resort and Spa,https://www.booking.com/hotel/us/miracle-sprin...,"10625 Palm Drive, Desert Hot Springs, CA 92240...",5.9,610,7.2,"2,125",0,NO
2,129,Joshua Tree National Park,High Desert Motel Joshua Tree National Park,https://www.booking.com/hotel/us/high-desert-m...,"61310 Twenty Nine Palms Highway, Joshua Tree, ...",5.9,349,7.4,"1,254",0,Yes
3,130,Joshua Tree National Park,A PLACE IN THE SUN Hotel - ADULTS ONLY Big Uni...,https://www.booking.com/hotel/us/a-place-in-th...,"754 East San Lorenzo Road, Palm Springs, CA 92...",5.9,651,7.5,698,0,Yes
4,131,Joshua Tree National Park,Korakia Pensione,https://www.booking.com/hotel/us/korakia-pensi...,"257 South Patencio, Palm Springs, CA 92262, Un...",5.9,"1,716",9.0,328,0,NO
...,...,...,...,...,...,...,...,...,...,...,...
120,247,Joshua Tree National Park,La Quinta Vacations Rental,https://www.booking.com/hotel/us/la-quinta-vac...,"50730 Santa Rosa Plaza, La Quinta, CA 92253, U...",5.9,656,8.8,212,0,NO
121,248,Joshua Tree National Park,SpringHill Suites by Marriott Palm Desert,https://www.booking.com/hotel/us/fairfield-inn...,"72-322 Highway 111, Palm Desert, 92260, United...",5.9,574,7.9,264,1,NO
122,249,Joshua Tree National Park,"L'Horizon Resort & Spa, Hermann Bungalows",https://www.booking.com/hotel/us/l-39-horizon-...,"1050 East Palm Canyon Drive, Palm Springs, CA ...",5.9,"2,200",8.7,163,0,Yes
123,250,Joshua Tree National Park,The Bliss House,https://www.booking.com/hotel/us/the-bliss-hou...,"1502 Amelia Way, Palm Springs, CA 92262, Unite...",5.9,"2,244",10,5,0,NO
